In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score as R2
from sklearn.metrics import mean_squared_error as MSE

In [131]:
rest = pd.read_csv('/Volumes/Database/Research/ozone-budget/result_df_UKESM1_3_0_NN_width_64_dropout_0.1_61.csv')
data = pd.read_csv('/Volumes/Database/Research/C7 Multivariate analysis/UKESM1-0-LL_diag_2014(1217).csv')

In [132]:
rest['tas'] = data['tas']
rest['pan'] = data['pan']
rest['photo1d'] = data['photo1d']

rest['o3_mole'] = rest['o3']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['no_mole'] = rest['no']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['no2_mole'] = rest['no2']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['oh_mole'] = rest['oh']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['h2o_mole'] = rest['h2o']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['hno3_mole'] = rest['hno3']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['pan_mole'] = rest['pan']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)

rest['ho2_mole'] = rest['HO₂']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['ch3o2_mole'] = rest['CH₃O₂']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)
rest['o1d_mole'] = rest['O¹D']*1.01325*10**5/rest['tas']*7.243*10**7*(10**9)

rest['k_ho2_ho2'] = 2.2*10**(-13)*np.exp(600/rest['tas'])
rest['k_oh_no2'] = 6.5*10**(-11)
rest['k_no_ho2'] = data['k_no_ho2']
rest['k_no_ch3o2'] = data['k_no_ch3o2']
rest['k_o1d_h2o'] = data['k_o1d_h2o']
rest['k_o3_ho2'] = data['k_o3_ho2']
rest['k_o3_oh'] = data['k_o3_oh']

In [133]:
rest['Pt_HO2_NO'] = rest['k_no_ho2']*rest['no_mole']*rest['ho2_mole']
rest['Pt_CH3O2_NO'] = rest['k_no_ch3o2']*rest['no_mole']*rest['ch3o2_mole']
rest['Pt_RO2_NO'] = rest['PO₃̂']-rest['Pt_HO2_NO']-rest['Pt_CH3O2_NO']

rest['Pt_O1D_H2O'] = rest['o1d_mole']*rest['h2o_mole']*rest['k_o1d_h2o']
rest['Pt_HO2_O3'] = rest['k_o3_ho2']*rest['ho2_mole']*rest['o3_mole']
rest['Pt_OH_O3'] = rest['k_o3_oh']*rest['oh_mole']*rest['o3_mole']
rest['Pt_VOCs_O3'] = rest['LO₃̂']-rest['Pt_O1D_H2O']-rest['Pt_HO2_O3']-rest['Pt_OH_O3']

In [134]:
country = pd.read_csv('/Users/csuen/Desktop/PhD/Maps/country_province_state_2*2.csv')
rest = rest.merge(country, how = 'left', left_on = ['longitude','latitude'], right_on = ['longitude','latitude'])
rest = rest.drop(['province','state'], axis=1)
rest_land = rest.dropna()

In [135]:
rest_land['PO₃̂'].mean(), rest_land['Pt_HO2_NO'].mean(), rest_land['Pt_CH3O2_NO'].mean(), rest_land['Pt_RO2_NO'].mean()

(2798949.9127303516, 1022508.2051784359, 1736834.465719418, 39607.24183251247)

In [136]:
rest_land['LO₃̂'].mean(), rest_land['Pt_O1D_H2O'].mean(), rest_land['Pt_HO2_O3'].mean(), rest_land['Pt_OH_O3'].mean(), rest['Pt_VOCs_O3'].mean()

(585368.0973117758,
 350394.80145394354,
 2029.1719433179328,
 67343.61033307547,
 150031.92553927866)

In [137]:
rest_land['Pt_VOCs_O3'].describe()

count    3.221400e+05
mean     1.656005e+05
std      2.841883e+05
min     -8.787243e+04
25%     -2.780083e+00
50%      1.372834e+04
75%      2.273214e+05
max      5.059831e+06
Name: Pt_VOCs_O3, dtype: float64